In [1]:
library(tidyverse)
library(wesanderson)
library(ggridges)
library(ggthemes)
library(reshape)
library(lme4)
library(coefplot2)
source("../../../spydr_scripts/functions/get_os.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape’

The following object is masked from ‘package:dplyr’:

    rename

The following objects are masked from ‘package:tidyr’:

    expand, smiths

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:reshape’:

    expand

The following object is masked from ‘package:tidyr’:

    expand

Loading required package: coda


In [2]:
os <- get_os()
if (os == 'linux'){home <- '/home/krista/'
} else if(os == 'osx'){home <- '/Users/67981492/'}

agg_data_path = paste0(home,'Dropbox/volatileValues/simple_rt_experiment_probabilityC/analysis/aggregated_data/')
setwd(agg_data_path)

In [3]:
clean_df <- read_csv('clean_df.csv',col_types = cols())
raw_df <- read_csv('raw_df_processed.csv',col_types = cols())
epoch_df <- read_csv('epoch_df.csv',col_types = cols())

(n_subjects = length(unique(clean_df$ID)))
(n_conditions = length(unique(clean_df$condition)))
n_clean_trials = 600

# #do this when complete 
# clean_df$cont_trial <- rep(seq(1,n_clean_trials),n_subjects*n_conditions)
# max(clean_df$cont_trial)
# head(clean_df); head(raw_df)

[1] 24

[1] 4

In [5]:
head(epoch_df)

LR_choice,obs_LR_accuracy,obs_LR_solution,reward,cumulative_reward,rt,total_trial_time,iti,cp_with_slow_fast,obs_cp_with_slow_fast,⋯,prop_112,n_epoch_trials,p_id_solution,obs_low_val_cue,id_choice,p_id_accuracy,ID_num,timepoint,conflict,vol
1,0,0,0,599,0.7115700,2.385458,0.6701990,1,0,⋯,0.3142857,35,111,111,111,1,761,1,high,mod
1,0,0,0,598,0.6814098,1.939019,0.2597073,0,1,⋯,0.3142857,35,111,112,112,0,761,1,high,mod
0,1,0,2,599,0.7671180,2.498060,0.7406146,0,0,⋯,0.3142857,35,111,112,111,1,761,1,high,mod
1,0,0,0,598,0.5072088,1.742433,0.2502416,0,0,⋯,0.3142857,35,111,112,112,0,761,1,high,mod
1,1,1,2,599,0.6562591,2.350417,0.7001898,0,0,⋯,0.3142857,35,111,112,111,1,761,1,high,mod
0,0,1,0,598,0.3423209,1.831150,0.4982311,0,1,⋯,0.3142857,35,111,111,111,1,761,1,high,mod


In [7]:
names(epoch_df)

[1] "LR_choice"             "obs_LR_accuracy"       "obs_LR_solution"      
 [4] "reward"                "cumulative_reward"     "rt"                   
 [7] "total_trial_time"      "iti"                   "cp_with_slow_fast"    
[10] "obs_cp_with_slow_fast" "obs_high_val_cue"      "ID"                   
[13] "condition"             "coded_condition"       "original_trial"       
[16] "epoch_n"               "epoch_trial"           "shifted_epoch_trial"  
[19] "max_epoch_n"           "repeat_trial"          "prop_112"             
[22] "n_epoch_trials"        "p_id_solution"         "obs_low_val_cue"      
[25] "id_choice"             "p_id_accuracy"         "ID_num"               
[28] "timepoint"             "conflict"              "vol"

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lc'| condition == 'hc')), 
       aes(shifted_epoch_trial, p_id_accuracy, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed") + 
       coord_fixed(10)

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lc'| condition == 'hc')), aes(shifted_epoch_trial, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition))  + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" )  + coord_fixed(50)

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lv'| condition == 'hv')), aes(shifted_epoch_trial, p_id_accuracy, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition))  + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" ) + 
       coord_fixed(10)

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lv'| condition == 'hv')), aes(shifted_epoch_trial, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition))  +  geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" )  + coord_fixed(50)

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lc'| condition == 'hc')), aes(shifted_epoch_trial, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition))  + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" ) + coord_fixed(50)

In [ ]:
options(repr.plot.width=30, repr.plot.height=20)

theme_set(theme_grey(base_size = 22)) 

ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lv'| condition == 'hv')), aes(shifted_epoch_trial, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) +  facet_wrap( ~ ID, ncol = 5) +  geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" ) +
ggtitle('Volatility: Reaction Times') + theme(plot.title = element_text(hjust = 0.5, size=40, face = 'bold'))

In [ ]:
# ggplot(subset(raw_df, max_e_trial >= 10 & shifted_epoch_trial < 10 & (condition == 'lv'| condition == 'hv')), aes(shifted_epoch_trial, rt, color=condition)) + 
# stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) +  facet_wrap( ~ ID, ncol = 5) + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" ) +
# ggtitle('Volatility: Accuracy') + theme(plot.title = element_text(hjust = 0.5, size=40, face = 'bold'))

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lc'| condition == 'hc')), aes(shifted_epoch_trial, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) +  facet_wrap( ~ ID, ncol = 5) + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" ) + 
ggtitle('Conflict: Reaction Times') + theme(plot.title = element_text(hjust = 0.5, size=40, face = 'bold'))

In [ ]:
# ggplot(subset(raw_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & (condition == 'lc'| condition == 'hc')), aes(shifted_epoch_trial, accuracy, color=condition)) + 
# stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) +  facet_wrap( ~ ID, ncol = 5) + geom_vline(xintercept=0, alpha = 0.5, size=1,linetype="dashed" ) + 
# ggtitle('Conflict: Accuracy') + theme(plot.title = element_text(hjust = 0.5, size=40, face = 'bold'))

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)

ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & epoch_n == max_epoch_n & (condition == 'lc'| condition == 'hc')), aes(shifted_epoch_trial, accuracy, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) 

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial < 10 & epoch_n == 2 & (condition == 'lc'| condition == 'hc')), aes(epoch_trial, rt, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) 

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & shifted_epoch_trial < 10 & epoch_n == 2 & (condition == 'lv'| condition == 'hv')), aes(shifted_epoch_trial, rt, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) 

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial < 10 & (condition == 'hc')), aes(trial, rt, color=condition)) +
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) + geom_line(aes(trial, cp_with_slow_fast), 'color' = 'black', alpha=0.2)

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial < 10 & (condition == 'hc')), aes(trial, accuracy, color=condition)) +
stat_summary(fun.data = "mean_cl_boot", geom = "smooth", aes(fill=condition)) + geom_line(aes(trial, cp_with_slow_fast), 'color' = 'black', alpha=0.2)

In [ ]:
max_epoch_n <- epoch_data %>% 
group_by(condition,ID) %>% summarize(epoch_n = max(epoch_n))

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10), aes(condition, accuracy, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('Accuracy')

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10), aes(condition, rt, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2) + ggtitle('Reaction Times')

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10), aes(condition, rt, fill=condition)) + geom_violin() + ggtitle('Reaction Times')

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10), aes(epoch_n, accuracy, color=condition)) + stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('Accuracy')

In [ ]:
#find the minimum number of epochs for each condition across subjects to ensure fair plotting 
min_epochs <- raw_df %>% group_by(condition, ID) %>% summarize(min_e = max(epoch_n)) %>% summarize(min_epochs=min(min_e))
min_epochs

In [ ]:
ggplot(subset(raw_df, n_epoch_trials >= 10 & epoch_trial <= 10 & epoch_n < 22 & (condition =='lc' | condition =='hc')), aes(epoch_n, accuracy, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('Accuracy') + scale_y_continuous(limits = c(0, 1))

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10 & epoch_n < 22 & (condition =='lc' | condition =='hc')), aes(epoch_n, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('RT')  

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10 & epoch_n < 15 & (condition =='lv' | condition =='hv')), aes(epoch_n, rt, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('RT') 

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10 & epoch_n < 16 & (condition =='lv')), aes(epoch_n, accuracy, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('Accuracy') + scale_y_continuous(limits = c(0, 1))

In [ ]:
ggplot(subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10 & epoch_n < 37 & (condition =='hv')), aes(epoch_n, accuracy, color=condition)) + 
stat_summary(fun.data = "mean_cl_boot", geom = "pointrange", size=2)  + ggtitle('Accuracy') +  scale_y_continuous(limits = c(0, 1))

In [ ]:
epoch_df <- subset(clean_df, n_epoch_trials >= 10 & epoch_trial <= 10) 

epoch_df[epoch_df$epoch_trial <= 5, 'timepoint'] = 1
epoch_df[epoch_df$epoch_trial > 5, 'timepoint'] = 2

epoch_df[epoch_df$condition == 'hc', 'conflict'] = 'high'
epoch_df[epoch_df$condition == 'lc', 'conflict'] = 'low'
epoch_df[epoch_df$condition == 'hc', 'vol'] = 'mod'
epoch_df[epoch_df$condition == 'lc', 'vol'] = 'mod'

epoch_df[epoch_df$condition == 'hv', 'vol'] = 'high'
epoch_df[epoch_df$condition == 'lv', 'vol'] = 'low'
epoch_df[epoch_df$condition == 'hv', 'conflict'] = 'mod'
epoch_df[epoch_df$condition == 'lv', 'conflict'] = 'mod'

epoch_df$conflict <- as.factor(epoch_df$conflict)
epoch_df$vol <- as.factor(epoch_df$vol)

# epoch_df$conflict <- relevel(epoch_df$conflict, ref = "low")
# epoch_df$vol <- relevel(epoch_df$vol, ref = "low")

# epoch_df$condition <- relevel(epoch_df$condition, ref = "lc")

In [ ]:
colnames(epoch_df)

In [ ]:
library(rstanarm)
#bayesian reg.
bayes_rt_reg <- stan_glmer(log(rt) ~ condition*timepoint + (1|ID), data=epoch_df, chains = 1)
bayes_acc_reg <- stan_glmer(accuracy ~ condition*timepoint + (1|ID), 
                 data=epoch_df, family=binomial, control = glmerControl(optimizer = "bobyqa"), chains = 1)

In [ ]:
rt_reg <- lmer(log(rt) ~ condition*timepoint + (1|ID), data=epoch_df)
acc_reg <- glmer(accuracy ~ condition*timepoint + (1|ID), 
                 data=epoch_df, family=binomial, control = glmerControl(optimizer = "bobyqa"))

summary(rt_reg)
summary(acc_reg)

In [ ]:
summary(rt_reg)

In [ ]:
library(sjPlot)
library(sjlabelled)
library(sjmisc)

plot_model(rt_reg, show.values = TRUE)

In [ ]:
plot_model(acc_reg, show.values = TRUE, transform="plogis")

In [ ]:
qqnorm(resid(rt_reg))
qqline(resid(rt_reg))

In [ ]:
plot(rt_reg)

In [ ]:
# (conf_rt_reg <-lmer(rt ~ conflict*timepoint + (1|ID), data=epoch_df)
# )
# (vol_rt_reg <-lmer(rt ~ vol*timepoint + (1|ID), data=epoch_df)
# )

# (conf_acc_reg <-glmer(accuracy ~ conflict*timepoint + (1|ID), 
#                  data=epoch_df, family=binomial, control = glmerControl(optimizer = "bobyqa"))
# )

# (vol_acc_reg <-glmer(accuracy ~ vol*timepoint + (1|ID), 
#                  data=epoch_df, family=binomial, control = glmerControl(optimizer = "bobyqa"))
# )

In [ ]:
# library(boot)
# rt_lm <- function(formula, data, indices) {
#   d <- data[indices,] 
#   fit <- lmer(formula, data=d)
#   return(fixef(fit))
# } 

# acc_log <- function(formula, data, indices) {
#   d <- data[indices,] 
#   fit <- glmer(formula, data=d, family=binomial,  control = glmerControl(optimizer = "bobyqa"))
#   return(fixef(fit))
# } 

In [ ]:
#do the bootstrapped estimates encompass 0? 

In [ ]:
# rt_formula <- rt ~ condition*timepoint + (1|ID) 
# acc_formula <- accuracy ~ condition*timepoint + (1|ID)

# rt_lm_boot <- boot(formula=rt_formula, data=epoch_df, statistic=rt_lm, 
#   	R=1000)
# acc_log_boot <- boot(formula=acc_formula, data=epoch_df, statistic=acc_log, 
#   	R=1000)

In [ ]:
# #odds ratios
# odds <-  exp(acc_log_boot$)
# p <- odds / (1 + odds)